In [80]:
import numpy as np
import threading
import random
import time
from skimage.transform import resize
from skimage.color import rgb2gray
from collections import deque

import gym
import pandas as pd
import tensorflow as tf
import tflearn

In [60]:
# https://www.oreilly.com/learning/introduction-to-reinforcement-learning-and-openai-gym

In [71]:
env = gym.make("Taxi-v2")

[2017-10-31 22:28:24,737] Making new env: Taxi-v2


In [72]:
env.reset()

284

In [73]:
env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [74]:
env.observation_space

Discrete(500)

In [76]:
env.action_space

Discrete(6)

In [49]:
env.env.s = 200
env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [56]:
# Step 0 - South
env.step(0)
env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)


In [59]:
state,reward,done,info = env.step(0)
print(state,reward,done,info)

132 -1 False {'prob': 1.0}


In [62]:
# Random Solver
state = env.reset()
counter = 0
reward = None
while reward != 20:
    state, reward, done, info = env.step(env.action_space.sample())
    counter += 1

print(counter)

2689


In [65]:
Q = np.zeros([env.observation_space.n, env.action_space.n])
G = 0
alpha = 0.618

for episode in range(1,1001):
    done = False
    G, reward = 0,0
    state = env.reset()
    while done != True:
            action = np.argmax(Q[state]) #1
            state2, reward, done, info = env.step(action) #2
            Q[state,action] += alpha * (reward + np.max(Q[state2]) - Q[state,action]) #3
            G += reward
            state = state2   
    if episode % 50 == 0:
        print('Episode {} Total Reward: {}'.format(episode,G))

Episode 50 Total Reward: -157
Episode 100 Total Reward: -73
Episode 150 Total Reward: -8
Episode 200 Total Reward: 14
Episode 250 Total Reward: 6
Episode 300 Total Reward: 12
Episode 350 Total Reward: 7
Episode 400 Total Reward: 12
Episode 450 Total Reward: -7
Episode 500 Total Reward: 8
Episode 550 Total Reward: 8
Episode 600 Total Reward: 10
Episode 650 Total Reward: 8
Episode 700 Total Reward: 9
Episode 750 Total Reward: 6
Episode 800 Total Reward: 8
Episode 850 Total Reward: 10
Episode 900 Total Reward: 10
Episode 950 Total Reward: 5
Episode 1000 Total Reward: 6


## PacMan
* https://github.com/tflearn/tflearn/blob/master/examples/reinforcement_learning/atari_1step_qlearning.py

In [66]:
env = gym.make("MsPacman-v0")
state = env.reset()

[2017-10-31 22:27:21,707] Making new env: MsPacman-v0


In [67]:
env.render()

In [68]:
env.action_space.n

9

In [69]:
env.env.get_action_meanings()

['NOOP',
 'UP',
 'RIGHT',
 'LEFT',
 'DOWN',
 'UPRIGHT',
 'UPLEFT',
 'DOWNRIGHT',
 'DOWNLEFT']

### Atari

In [30]:
env = gym.make('SpaceInvaders-v0')
observation_n = env.reset() #retrieve observation

[2017-10-31 22:04:39,528] Making new env: SpaceInvaders-v0


In [21]:
env.render()

In [31]:
env.observation_space

Box(210, 160, 3)

In [17]:
# while True:
#     action_n = [[('KeyEvent','ArrowLeft',True)] for ob in observation_n]

In [15]:
# observation_n - observation of environment
# reward_n : if action was beneficial
# done_n : Indicates if the game is over or not: Yes/No
# info : Additional Info .. 

observation_n, reward_n, done_n, info = env.step(action_n)


array([80, 89, 22], dtype=uint8)

In [4]:
for _ in range(100):
    env.render()
    env.step(env.action_space.sample())
    